<div style="line-height:1.2;">

<h1 style="color:#BF66F2; margin-bottom: 0.3em;">PyTorch basics 2</h1>

<h4 style="margin-top: 0.3em; margin-bottom: 1em;"> Create models, save and load them via checkpoints. Focus on ONNX engine. </h4>

<div style="line-height:1.4; margin-bottom: 0.5em;">
    <h3 style="color: lightblue; display: inline; margin-right: 0.5em;">Keywords:</h3> 
    device check without warnings + onnxruntime + torchvision models
</div>

</div>

In [10]:
import torch
import warnings 
import numpy as np
import torchvision
import torch.optim as optim  

import onnxruntime

In [11]:
"""  Check if CUDA is available """
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [12]:
## Create a tensor on the GPU
x = torch.randn(3, 3, device=device)

## Move the tensor to the CPU
x_cpu = x.to('cpu')

## Control the type and device of the tensors
print(f'Tensor x type: {x.type()}, device: {x.device}')
print(f'Tensor x_cpu type: {x_cpu.type()}, device: {x_cpu.device}')

Tensor x type: torch.FloatTensor, device: cpu
Tensor x_cpu type: torch.FloatTensor, device: cpu


<h3 style="color:#BF66F2 ">  => First way </h3>

In [13]:
def save_checkpoint(state, filename="./checkpoints/my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [14]:
model = torchvision.models.vgg16(weights=None)      #pretrained=False deprecated!
optimizer = optim.Adam(model.parameters())
checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}

save_checkpoint(checkpoint)
load_checkpoint(torch.load("./checkpoints/my_checkpoint.pth.tar"), model, optimizer)

=> Saving checkpoint
=> Loading checkpoint


In [ ]:
epoch = 10 
checkpoint = {"epoch": epoch, "model_state_dict": model.state_dict(), "optimizer_state_dict": optimizer.state_dict()}
torch.save(checkpoint, "./checkpoints/checkpoint2.pth")

In [15]:
checkpoint = torch.load("./checkpoints/checkpoint2.pth")
model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
epoch = checkpoint["epoch"]

<h3 style="color:#BF66F2 ">  => Second way </h3>

<h3 style="color:#BF66F2"> Recap: Open Neural Network Exchange</h3>
<div style="margin-top: -4px;">
ONNX Runtime is a scoring engine to store ML and DL models in ONNX open format. <br>
It provides a shared model representation that enables interoperability between various AI frameworks. 
</div>

In [26]:
"""  Export an initialized PyTorch model to the Open Neural Network Exchange (ONNX) format.
Then, load the exported ONNX model (using ONNX Runtime) and perform inference.
N.B. 
# To avoid UserWarning related to "pretrained=False" option, which is deprecated!
=> model = torchvision.models.vgg16(pretrained=False)
"""
model = torchvision.models.vgg16(weights=None)

batch_size = 3
input_size = 100
# Export the PyTorch model to the ONNX format, with a shape suitable for VGG16 (3 channels, 224x224 pixels).
input_tensor = torch.randn(batch_size, 3, 224, 224)
# Resize the input tensor to 224x224 using bicubic interpolation
input_tensor_resized = torch.nn.functional.interpolate(input_tensor, size=(224, 224), mode='bicubic', align_corners=False)
torch.onnx.export(model, input_tensor_resized, "./checkpoints/modelonnxsaved.onnx")

# Load the ONNX model
session = onnxruntime.InferenceSession("./checkpoints/modelonnxsaved.onnx")
inputs = {"input.1": input_tensor_resized.numpy()}    #input.1 ! 

outputs = session.run(None, inputs)
output_tensor = torch.from_numpy(outputs[0])

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

